In [82]:
import itertools

from extract import extract_dataset_archive
from split import convert_train_set_to_images
import os
from tensorflow.keras.layers import Dense, LSTM, Activation
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import minmax_scale
# os.system('python scripts/demo_inference.py --cfg pretrained_models/256x192_res152_lr1e-3_1x-duc.yaml --checkpoint  pretrained_models/fast_421_res152_256x192.pth --video ../dataset/NonViolence/NV_1.mp4 --outdir ../dataset --sp')

def prepare_dataset():
    extract_dataset_archive()
    convert_train_set_to_images()


def convert_videos_to_json(path_in, path_out):
    os.system('python scripts/demo_inference.py --cfg pretrained_models/256x192_res152_lr1e-3_1x-duc.yaml '
              '--checkpoint  pretrained_models/fast_421_res152_256x192.pth --video ' + path_in + ' --outdir ' + path_out + ' --sp')


#prepare_dataset()
#os.chdir("AlphaPose")

# for i in range(1, 1001):
#    file_in = str("../dataset/Violence/V_") + str(i) + ".mp4"
#    file_out = str("../dataset/results/V_") + str(i) + "_json"
#    convert_videos_to_json(file_in, file_out)

#for i in range(1, 1001):
#    file_in = str("../dataset/NonViolence/NV_") + str(i) + ".mp4"
#    file_out = str("../dataset/results/NV_") + str(i) + "_json"
#    convert_videos_to_json(file_in, file_out)



person_count = 5
max_frames = 280


ModuleNotFoundError: No module named 'sorted_containers'

In [131]:
model = Sequential()
model.add(LSTM(100, input_shape=(5, 68))) # 150, 2048 = n_chunks, chunk_size
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(50))
model.add(Activation('sigmoid'))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

csv_dir = "dataset/csvs"

csv_files = os.listdir(csv_dir)

train_files, test_files = train_test_split(csv_files, train_size=0.8)

Instructions for updating:
Use tf.cast instead.


In [142]:
for file_name in train_files:
    csv = pd.read_csv(csv_dir + "/" + file_name)
    
    violent = csv.violent[0]
    
    # Resize all CSV files to same arbitrary limit
    csv = csv.drop('violent',axis=1)
    
    labels = csv.columns.values
    
    
    for label in labels[2:]:
        minmax_scale = preprocessing.MinMaxScaler().fit(csv[[label]])
        csv[[label]] = minmax_scale.transform(csv[[label]])
    
#     column_trans = ColumnTransformer(
#         [('scaler', MinMaxScaler(),list(range(2,70)))],
#         remainder='passthrough') 
#     column_trans.fit_transform(csv)
    
    data = csv.to_numpy()

    val_cumulative = {}
    frame_ids = set()
    for person in data:
        frame_ids.add(person[0])
        frame, person_id = person[:2]

        vel_array = []
        for vel in range(5, 70, 4):
            vel_array.append(vel)

        total_vel = sum(vel_array)

        if person_id in val_cumulative:
            val_cumulative[person_id] += total_vel
        else:
            val_cumulative[person_id] = 0
        
        
    val_cumulative = {k: v for k, v in sorted(val_cumulative.items(), key=lambda item: item[1], reverse=True)}
    top_people = dict(itertools.islice(val_cumulative.items(), person_count))
    top_people_dataset = {}
    
    current_frame = 0
    for frame_id in frame_ids:
        # Trim to 280 if above
        if current_frame == max_frames:
            break
        top_people_dataset[frame_id] = []
        people = np.array([item for item in data if item[0] == frame_id and item[1] in top_people.keys()])
        fill_amount = person_count
        for person in people:
            top_people_dataset[frame_id].append(person[2:])
            fill_amount -= 1
        for i in range(0, fill_amount):
            top_people_dataset[frame_id].append(np.zeros(68))
        current_frame += 1
            
    
    # Pad to 280 frames if below
    for i in range(current_frame+1, max_frames+1):
        top_people_dataset[i] = []
        for j in range(0, person_count):
            top_people_dataset[i].append(np.zeros(68))
                 
    x_train = np.asarray(list(top_people_dataset.values()))
    
    if violent == 0:
        y_train = np.zeros(len(x_train))
    else:
        y_train = np.ones(len(x_train))
    
#     x_train = np.array(csv.drop(['frame', 'object_id', 'violent'],axis=1))
    print(model.train_on_batch(x_train, y_train))
#     pd.DataFrame(history.history).plot()

[0.11111435, 0.0]
[0.11111155, 0.007142857]
[0.11111136, 0.88928574]
[0.11111177, 0.9]
[0.11111191, 0.91071427]
[0.11111213, 0.91785717]
[0.11111271, 0.825]
[0.111112475, 0.0]


In [143]:
for file_name in train_files:
    csv = pd.read_csv(csv_dir + "/" + file_name)
    
    violent = csv.violent[0]
    
    # Resize all CSV files to same arbitrary limit
    csv = csv.drop('violent',axis=1)
    
    labels = csv.columns.values
    
    
    for label in labels[2:]:
        minmax_scale = preprocessing.MinMaxScaler().fit(csv[[label]])
        csv[[label]] = minmax_scale.transform(csv[[label]])
    
#     column_trans = ColumnTransformer(
#         [('scaler', MinMaxScaler(),list(range(2,70)))],
#         remainder='passthrough') 
#     column_trans.fit_transform(csv)
    
    data = csv.to_numpy()

    val_cumulative = {}
    frame_ids = set()
    for person in data:
        frame_ids.add(person[0])
        frame, person_id = person[:2]

        vel_array = []
        for vel in range(5, 70, 4):
            vel_array.append(vel)

        total_vel = sum(vel_array)

        if person_id in val_cumulative:
            val_cumulative[person_id] += total_vel
        else:
            val_cumulative[person_id] = 0
        
        
    val_cumulative = {k: v for k, v in sorted(val_cumulative.items(), key=lambda item: item[1], reverse=True)}
    top_people = dict(itertools.islice(val_cumulative.items(), person_count))
    top_people_dataset = {}
    
    current_frame = 0
    for frame_id in frame_ids:
        # Trim to 280 if above
        if current_frame == max_frames:
            break
        top_people_dataset[frame_id] = []
        people = np.array([item for item in data if item[0] == frame_id and item[1] in top_people.keys()])
        fill_amount = person_count
        for person in people:
            top_people_dataset[frame_id].append(person[2:])
            fill_amount -= 1
        for i in range(0, fill_amount):
            top_people_dataset[frame_id].append(np.zeros(68))
        current_frame += 1
            
    
    # Pad to 280 frames if below
    for i in range(current_frame+1, max_frames+1):
        top_people_dataset[i] = []
        for j in range(0, person_count):
            top_people_dataset[i].append(np.zeros(68))
                 
    x_test = np.asarray(list(top_people_dataset.values()))
    
    if violent == 0:
        y_test= np.zeros(len(x_test))
    else:
        y_test = np.ones(len(x_test))


    print(model.test_on_batch(x_test, y_test))
# model.fit(x_train, y_train, epochs, batch_size=batchs, verbose=1)
# model.save("rnn.h5", overwrite=True)

[0.111112155, 0.05]
[0.11111246, 0.0]
[0.11111248, 0.0]
[0.1111123, 0.0]
[0.11111256, 0.0]
[0.1111124, 0.0]
[0.111112714, 0.0035714286]
[0.11111256, 0.0]


In [88]:
output.shape

(124, 5, 68)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])